# 🏆 Final Training Notebook
## Train, evaluate, and save your best model

This notebook brings everything together for the final training run.

## 1. Setup

In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Import your modules
from config import *
from src.utils import set_seed, check_gpu

# Setup
set_seed()
check_gpu()

## 2. Load Data

In [ ]:
# TODO: Import and use your dataset
# from src.dataset import ArabicLetterDataset, get_transforms, create_data_loaders

# transform = get_transforms()
# full_dataset = ArabicLetterDataset(...)
# train_loader, val_loader = create_data_loaders(...)

print("TODO: Load your dataset here")

## 3. Create Model

In [ ]:
# TODO: Import and create your model
# from src.model import ArabicCNN, print_model_summary

# model = ArabicCNN().to(DEVICE)
# print_model_summary(model)

print("TODO: Create your model here")

## 4. Train

In [ ]:
# TODO: Run training
# from src.train import train_model

# history = train_model(
#     model=model,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     num_epochs=NUM_EPOCHS,
#     learning_rate=LEARNING_RATE,
#     device=DEVICE
# )

print("TODO: Train your model here")

## 5. Plot Training History

In [ ]:
# TODO: Plot results
# from src.evaluate import plot_training_history
# plot_training_history(history, '../outputs/training_history.png')

print("TODO: Plot training history here")

## 6. Evaluate

In [ ]:
# TODO: Evaluate model
# from src.evaluate import evaluate_model, plot_confusion_matrix, get_classification_report

# predictions, labels, accuracy = evaluate_model(model, val_loader, DEVICE)
# get_classification_report(labels, predictions)
# plot_confusion_matrix(labels, predictions, save_path='../outputs/confusion_matrix.png')

print("TODO: Evaluate your model here")

## 7. Save Final Model

In [ ]:
# TODO: Save model
# torch.save(model.state_dict(), '../models/final_model.pth')
# print("Model saved!")

print("TODO: Save your model here")

## 8. Test on Your Own Images!

Try writing an Arabic letter and testing it!

In [ ]:
# TODO: Test with your own image
# from PIL import Image
# from src.evaluate import predict_single_image

# # Load your image
# my_image = Image.open('path/to/your/image.png')
# my_image_tensor = transform(my_image)

# # Predict
# predicted_class, confidence, probs = predict_single_image(model, my_image_tensor, DEVICE)

print("TODO: Test with your own handwritten letters!")

---

# 🎉 Congratulations!

You've built an AI that can recognize handwritten Arabic letters!

**What you learned:**
- How to load and preprocess image data
- How to build a Convolutional Neural Network
- How to train a model with PyTorch
- How to evaluate model performance

**Next challenges:**
- Can you get accuracy above 90%?
- Try adding more layers or data augmentation
- Build a simple web interface to demo your model